In [1]:
import re
import pickle
import random as rd
import pandas as pd
import torch.nn as nn
import seaborn as sns
from nltk.corpus import stopwords
from tokenizers import Tokenizer,models,trainers
from sklearn.model_selection import train_test_split

In [3]:
lenta_news = pd.read_csv('lenta_news.csv')
lenta_news

,title,text,topic
0,МВД создало бюро по борьбе с отмыванием денег,В МВД России создано оперативно-розыскное бюро...,Экономика
1,Правительство не будет занимать деньги у ЦБ,Правительство не будет прибегать к заимствован...,Экономика
2,"ВВС США заказали 60 ""Боингов"" для вторжения в ...",Компания Boeing получит пятилетний контракт на...,Экономика
3,Правительственная комиссия не дала ExxonMobil ...,Результаты конкурса на право освоения месторож...,Экономика
4,Неимущим для получения пособия придется расста...,В доходах малоимущих россиян будут учитываться...,Экономика
...,...,...,...
23944,"Борис Гребенщиков награжден орденом ""За заслуг...","Лидер легендарной группы ""Аквариум"" Борис Греб...",Культура
23945,Мадонна и Траволта названы худшими актерами года,"24 марта, накануне вручения премии Американско...",Культура
23946,Габриэль Гарсиа Маркес простил Голливуд ради д...,Габриэль Гарсиа Маркес,Культура
23947,Аль Пачино отстоял в суде свое право на детей,Знаменитый американский актер Аль Пачино добил...,Культура


In [31]:
rbk_news = pd.read_csv('rbk_news.csv',header = None)
rbk_news

,0,1
0,США обеспокоены участием военных КНДР в военно...,Политика
1,"Пхеньян, по ее словам «не помогает» в урегулир...",Политика
2,США «по-прежнему обеспокоены прямым участием» ...,Политика
3,«Переброска на территорию России военных [КНДР...,Политика
4,\n\n\n\n\n\n\n Минобороны п...,Политика
...,...,...
1170,\n\n\n\n\n\n\n ЦБ подумает ...,Финансы
1171,В отличие от стратегий автоследования в ОПИФе ...,Финансы
1172,\n\n\n\n\n\n\n Индекс Мосби...,Финансы
1173,Идею привлечения авторов инвестиционных блогов...,Финансы


In [17]:
rbk_news.iloc[:,1].unique()

array(['Политика', 'Экономика', 'Бизнес', 'Технологии и медиа', 'Финансы'],
      dtype=object)

In [16]:
lenta_news.iloc[:,2].unique()

array(['Экономика', 'Интернет и СМИ', 'Спорт', 'Культура'], dtype=object)

In [37]:
rbk_news.loc[rbk_news[1] == 'Технологии и медиа',1] = 'Интернет и СМИ'

In [42]:
rbk_news[1].unique()

array(['Политика', 'Экономика', 'Бизнес', 'Интернет и СМИ', 'Финансы'],
      dtype=object)

In [46]:
lenta_news.iloc[:,2].unique()

array(['Экономика', 'Интернет и СМИ', 'Спорт', 'Культура'], dtype=object)

In [49]:
all_classes = list(set(rbk_news[1].unique().tolist() + lenta_news.iloc[:,2].unique().tolist()))

In [51]:
print(len(all_classes))
all_classes

7


['Экономика',
 'Бизнес',
 'Финансы',
 'Культура',
 'Интернет и СМИ',
 'Спорт',
 'Политика']

In [65]:
lent_text_col = pd.DataFrame({'Text':[lenta_news.iloc[row,0]+ ' ' +lenta_news.iloc[row,1] for row in range(len(lenta_news))]})

In [ ]:
lenta_news.insert(loc = 0,column = 'Text',value = lent_text_col['Text'])

In [80]:
lenta_news.drop(['title','text'],axis = 1,inplace = True)

In [81]:
lenta_news

,Text,topic
0,МВД создало бюро по борьбе с отмыванием денег ...,Экономика
1,Правительство не будет занимать деньги у ЦБ Пр...,Экономика
2,"ВВС США заказали 60 ""Боингов"" для вторжения в ...",Экономика
3,Правительственная комиссия не дала ExxonMobil ...,Экономика
4,Неимущим для получения пособия придется расста...,Экономика
...,...,...
23944,"Борис Гребенщиков награжден орденом ""За заслуг...",Культура
23945,Мадонна и Траволта названы худшими актерами го...,Культура
23946,Габриэль Гарсиа Маркес простил Голливуд ради д...,Культура
23947,Аль Пачино отстоял в суде свое право на детей ...,Культура


In [83]:
rbk_news.columns = ['Text','topic']

In [84]:
rbk_news

,Text,topic
0,США обеспокоены участием военных КНДР в военно...,Политика
1,"Пхеньян, по ее словам «не помогает» в урегулир...",Политика
2,США «по-прежнему обеспокоены прямым участием» ...,Политика
3,«Переброска на территорию России военных [КНДР...,Политика
4,\n\n\n\n\n\n\n Минобороны п...,Политика
...,...,...
1170,\n\n\n\n\n\n\n ЦБ подумает ...,Финансы
1171,В отличие от стратегий автоследования в ОПИФе ...,Финансы
1172,\n\n\n\n\n\n\n Индекс Мосби...,Финансы
1173,Идею привлечения авторов инвестиционных блогов...,Финансы


In [96]:
print(rbk_news.columns)
print(lenta_news.columns)
rbk_news.columns == lenta_news.columns

Index(['Text', 'topic'], dtype='object')
Index(['Text', 'topic'], dtype='object')


array([ True,  True])

In [97]:
all_news_data = pd.concat([lenta_news,rbk_news],axis = 0,ignore_index=True)

In [100]:
all_news_data

,Text,topic
0,МВД создало бюро по борьбе с отмыванием денег ...,Экономика
1,Правительство не будет занимать деньги у ЦБ Пр...,Экономика
2,"ВВС США заказали 60 ""Боингов"" для вторжения в ...",Экономика
3,Правительственная комиссия не дала ExxonMobil ...,Экономика
4,Неимущим для получения пособия придется расста...,Экономика
...,...,...
25119,\n\n\n\n\n\n\n ЦБ подумает ...,Финансы
25120,В отличие от стратегий автоследования в ОПИФе ...,Финансы
25121,\n\n\n\n\n\n\n Индекс Мосби...,Финансы
25122,Идею привлечения авторов инвестиционных блогов...,Финансы


In [103]:
labels = all_news_data['topic'].unique()
print(len(labels))
labels

7


array(['Экономика', 'Интернет и СМИ', 'Спорт', 'Культура', 'Политика',
       'Бизнес', 'Финансы'], dtype=object)

In [99]:
all_news_data.to_csv('all_news_data.csv')

In [117]:
for one_label in labels:
    the_one = all_news_data.loc[all_news_data['topic'] == one_label,'Text'].iloc[rd.randint(0,300)]
    print(one_label)
    print(the_one)
    print('\n\n')

Экономика
Касьянов заинтересовал Словакию новым газопроводом Словацкая сторона проявила "заметный интерес" к идее строительства газопровода по своей территории для транспортировки российского газа в Европу, об этом заявлено на встрече премьер-министра РФ Михаила Касьянова и министра экономики Словакии Любомира Гараха в среду в московском Доме правительства. Как сообщают из Департамента правительственной информации, Словакия уже заявила, что придает большое значение этому проекту, который свяжет магистральные газопроводы на территории Словакии и Польши. Сооружение новой нитки газопровода через Словакию позволит направить российский газ с Ямала в Европу в обход Украины, которая продолжает прибегать к несанкционированным заборам российского газа. Потери России от действий Украины оцениваются в несколько миллиардов кубических  метров в год. Беседа Касьянова и Гараха состоялась в рамках обсуждения итогов прошедшего накануне заседания российско-словацкой межправительственной комиссии по науч

In [9]:
all_news = pd.read_csv('all_news_data.csv',index_col = 0)

In [138]:
Train_news,Test_news = train_test_split(all_news,stratify=all_news['topic'],train_size=0.93)

In [139]:
print(len(Train_news))
print(len(Test_news))

23365
1759


In [ ]:
# Train_news.to_csv('Train_news.csv')
# Test_news.to_csv('Test_news.csv')

In [2]:
Train_news,Test_news = pd.read_csv('Train_news.csv',index_col=0),pd.read_csv('Test_news.csv',index_col=0)

In [ ]:
# Test_news = Test_news.reset_index().drop(['index'],axis = 1)
# Train_news = Train_news.reset_index().drop(['index'],axis = 1)

In [3]:
print(len(Train_news))
print(len(Test_news))

23364
1759


In [6]:
print(len(Train_news))
Train_news.iloc[:5,:]

23364


,Text,topic
0,"Украина готова вернуть ""взятый по ошибке"" кред...",Экономика
1,Рио-де-Жанейро стал кандидатом на роль столицы...,Спорт
2,Скончался драматург и сценарист Александр Воло...,Культура
3,Испанец пересек Атлантический океан на водном ...,Спорт
4,"Украина на Новый год останавливает все, что дв...",Интернет и СМИ


In [7]:
print(len(Test_news))
Test_news.iloc[:5,:]

1759


,Text,topic
0,"Падение фондового рынка возглавили ""ЮКОС"" и ""Г...",Экономика
1,Контрабанда нефти из Ирака достигла рекордного...,Экономика
2,"Андрей Шевченко продлил контракт с ""Миланом"" д...",Спорт
3,Американцы запрещают детям играть в компьютерн...,Интернет и СМИ
4,Napster идет маршем на Вашингтон Napster призы...,Интернет и СМИ


In [8]:
len(Test_news)

1759

In [13]:
len(Test_news.dropna())

1759

In [ ]:
# Train_news.dropna(inplace=True)
# Train_news.reset_index().drop('index',axis = 1,inplace = True)

In [ ]:
# Train_news.to_csv('Train_news.csv')

In [14]:
russian_stopwords = stopwords.words('russian')
print(len(russian_stopwords))
russian_stopwords[:10]

151


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [16]:
def normalize_text(the_text):
    lowered = the_text.lower()
    found = re.findall(r'[А-я0-9A-z]+',lowered)
    filtered_found = [word for word in found if word != '' and word not in russian_stopwords]
    return filtered_found

In [20]:
example_text = Train_news.iloc[3,0]
example_text

'Испанец пересек Атлантический океан на\xa0водном мотоцикле Испанец Альваро де Маричалар (Alvaro de Marichalar) пересек Атлантический океан на водном мотоцикле, сообщает в воскресенье BBC News. Маричалар начал свое путешествие четыре месяца в Риме. Проплыв по реке Тибр до Средиземного моря, он делал остановки в Марокко, на Гибралтаре и на Канарских островах. Достигнув Атлантического океана, путешественник посетил Кубу и в Доминиканскую республику. Конечной точкой маршрута Маричалара стал Майами, США. Маричалар ехал 12 часов в сутки. На протяжении всего маршрута его сопровождало судно с командой из шести человек на борту, которое перевозило еду и топливо для водного мотоцикла. По словам испанца, он потратил на подготовку к путешествию несколько лет, и поправился на 18 килограмм, чтобы компенсировать потерю веса во время прохождения маршрута. Отвечая на вопрос журналистов, что было самым тяжелым во время пересечения Атлантического океана, Маричалар ответил: "Холод. По дороге из Рима в Не

In [21]:
def cleared_texts(text_col):
    for ith_row in range(len(text_col)):
        for token in normalize_text(text_col.iloc[ith_row]):
            yield token

In [22]:
tokens_dataset = cleared_texts(Train_news['Text'])

In [23]:
wordlevel_trainer = trainers.WordLevelTrainer(vocab_size = 500_000,show_progress = True,special_tokens = ['[UNK]','[PAD]'],min_frequency = 5)
wordlevel_model = models.WordLevel()
Train_tokenizer = Tokenizer(model = wordlevel_model)

In [24]:
Train_tokenizer.train_from_iterator(tokens_dataset,trainer=wordlevel_trainer)

In [ ]:
# Train_tokenizer.get_vocab_size()

57568

In [ ]:
# with open('Train_tokenizer.pkl','wb') as file:
#     pickle.dump(Train_tokenizer,file)

In [26]:
with open('Train_tokenizer.pkl','rb') as file:
    Train_tokenizer = pickle.load(file)

In [28]:
Train_tokenizer.get_vocab_size()

57568

In [65]:
test_tokens_dataset = cleared_texts(Test_news['Text'])

In [66]:
wordlevel_trainer = trainers.WordLevelTrainer(vocab_size = 500_000,show_progress = True,special_tokens = ['[UNK]','[PAD]'])
wordlevel_model = models.WordLevel()
Test_tokenizer = Tokenizer(model = wordlevel_model)

In [67]:
Test_tokenizer.train_from_iterator(test_tokens_dataset,trainer=wordlevel_trainer)

In [75]:
Test_tokenizer.get_vocab_size()

51219

In [ ]:
# with open('Test_tokenizer.pkl','wb') as file:
#     pickle.dump(Test_tokenizer,file)

In [78]:
with open('Test_tokenizer.pkl','rb') as file:
    Test_tokenizer = pickle.load(file)

In [79]:
Test_tokenizer.get_vocab_size()

51219